In [2]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.ensemble
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolDescriptors as desc
from rdkit.DataStructs import cDataStructs as ds
from rdkit.DataStructs import UIntSparseIntVect as usv


In [59]:
testset = pd.read_csv('orig_test.csv')
smiles_test = testset['smiles']
testID = testset['Id']

In [50]:
data = pd.read_csv('orig_train.csv',nrows=100000)
nfeats = 400
feat_comp = data[:nfeats]
comps = feat_comp['smiles']
mid = (data.shape[0]-nfeats)/2
train = data[nfeats+mid:]
smiles_t = train['smiles']
valid = data[:nfeats+mid]
smiles_v = valid['smiles']
ytrain = train['gap']
yvalid = valid['gap']
train = train.drop(['smiles','gap'],1)
valid = valid.drop(['smiles','gap'],1)

In [51]:
from rdkit.Chem import MACCSkeys, DataStructs
from rdkit.Chem.Fingerprints import FingerprintMols

In [52]:
comps_vec = []
count = 0
for smiles in comps:
    m = Chem.MolFromSmiles(smiles)
    keys = MACCSkeys.GenMACCSKeys(m)
    comps_vec.append(keys)
print "Done."

Done.


In [53]:
train_keys = []
for smiles in smiles_t:
    m = Chem.MolFromSmiles(smiles)
    keys = MACCSkeys.GenMACCSKeys(m)
    train_keys.append(keys)
print "Done."

Done.


In [54]:
xtrain = pd.DataFrame(np.ones(len(train_keys)))
i = 0
for comp in comps_vec:
    sims = []
    for key in train_keys:
        sim = DataStructs.FingerprintSimilarity(comp,key)
        sims.append(sim)
    xtrain[comp] = sims

In [55]:
# valid_keys = []
# for smiles in smiles_v:
#     m = Chem.MolFromSmiles(smiles)
#     keys = MACCSkeys.GenMACCSKeys(m)
#     valid_keys.append(keys)
# print "Done."

Done.


In [ ]:
test_keys = []
for smiles in smiles_test:
    m = Chem.MolFromSmiles(smiles)
    keys = MACCSkeys.GenMACCSKeys(m)
    test_keys.append(keys)
print "Done."

In [56]:
# xvalid = pd.DataFrame(np.ones(len(valid_keys)))
# for comp in comps_vec:
#     sims = []
#     for key in valid_keys:
#         sim = DataStructs.FingerprintSimilarity(comp,key)
#         sims.append(sim)
#     xvalid[comp] = sims

In [ ]:
xtest = pd.DataFrame(np.ones(len(test_keys)))
for comp in comps_vec:
    sims = []
    for key in test_keys:
        sim = DataStructs.FingerprintSimilarity(comp,key)
        sims.append(sim)
    xtest[comp] = sims

In [57]:
rf = sklearn.ensemble.RandomForestRegressor()
rf.fit(xtrain,ytrain)
pred = rf.predict(xtest) 


In [ ]:
pred.to_csv("preds_MACCS.csv",index=None)

In [58]:
error = mean_squared_error(yvalid,pred)
error

0.065621754243096603